In [9]:
import pyarrow as pa
import pyarrow.ipc as ipc

# Load the .arrow file
with pa.memory_map("scripts/passengers.arrow", "r") as source:
    reader = ipc.RecordBatchFileReader(source)
    table = reader.read_all()

In [10]:
# Print column names to understand the data structure
print(table.column_names)

# Example: Extracting a specific column (e.g., 'target')
target_data = table.column('target').to_pylist()

# If there's a 'start' or time column, you can extract it similarly
time_data = table.column('start').to_pylist()


['start', 'target']


In [14]:
print(len(target_data[0]))
target_data[0]

144


[112,
 118,
 132,
 129,
 121,
 135,
 148,
 148,
 136,
 119,
 104,
 118,
 115,
 126,
 141,
 135,
 125,
 149,
 170,
 170,
 158,
 133,
 114,
 140,
 145,
 150,
 178,
 163,
 172,
 178,
 199,
 199,
 184,
 162,
 146,
 166,
 171,
 180,
 193,
 181,
 183,
 218,
 230,
 242,
 209,
 191,
 172,
 194,
 196,
 196,
 236,
 235,
 229,
 243,
 264,
 272,
 237,
 211,
 180,
 201,
 204,
 188,
 235,
 227,
 234,
 264,
 302,
 293,
 259,
 229,
 203,
 229,
 242,
 233,
 267,
 269,
 270,
 315,
 364,
 347,
 312,
 274,
 237,
 278,
 284,
 277,
 317,
 313,
 318,
 374,
 413,
 405,
 355,
 306,
 271,
 306,
 315,
 301,
 356,
 348,
 355,
 422,
 465,
 467,
 404,
 347,
 305,
 336,
 340,
 318,
 362,
 348,
 363,
 435,
 491,
 505,
 404,
 359,
 310,
 337,
 360,
 342,
 406,
 396,
 420,
 472,
 548,
 559,
 463,
 407,
 362,
 405,
 417,
 391,
 419,
 461,
 472,
 535,
 622,
 606,
 508,
 461,
 390,
 432]

In [7]:
time_data

[datetime.datetime(2000, 1, 1, 0, 0)]

In [58]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/AileenNielsen/TimeSeriesAnalysisWithPython/master/data/AirPassengers.csv")



In [42]:
from pathlib import Path
from typing import List, Union

import numpy as np
from gluonts.dataset.arrow import ArrowWriter

def convert_to_arrow(
    path: Union[str, Path],
    time_series: Union[List[np.ndarray], np.ndarray],
    compression: str = "lz4",
):
    """
    Store a given set of series into Arrow format at the specified path.

    Input data can be either a list of 1D numpy arrays, or a single 2D
    numpy array of shape (num_series, time_length).
    """
    assert isinstance(time_series, list) or (
        isinstance(time_series, np.ndarray) and
        time_series.ndim == 2
    )

    # Set an arbitrary start time
    start = np.datetime64("2000-01-01 00:00", "s")

    dataset = [
        {"start": start, "target": ts} for ts in time_series
    ]

    ArrowWriter(compression=compression).write_to_file(
        dataset,
        path=path,
    )


In [53]:
[df['#Passengers'].values.tolist()]

[[112,
  118,
  132,
  129,
  121,
  135,
  148,
  148,
  136,
  119,
  104,
  118,
  115,
  126,
  141,
  135,
  125,
  149,
  170,
  170,
  158,
  133,
  114,
  140,
  145,
  150,
  178,
  163,
  172,
  178,
  199,
  199,
  184,
  162,
  146,
  166,
  171,
  180,
  193,
  181,
  183,
  218,
  230,
  242,
  209,
  191,
  172,
  194,
  196,
  196,
  236,
  235,
  229,
  243,
  264,
  272,
  237,
  211,
  180,
  201,
  204,
  188,
  235,
  227,
  234,
  264,
  302,
  293,
  259,
  229,
  203,
  229,
  242,
  233,
  267,
  269,
  270,
  315,
  364,
  347,
  312,
  274,
  237,
  278,
  284,
  277,
  317,
  313,
  318,
  374,
  413,
  405,
  355,
  306,
  271,
  306,
  315,
  301,
  356,
  348,
  355,
  422,
  465,
  467,
  404,
  347,
  305,
  336,
  340,
  318,
  362,
  348,
  363,
  435,
  491,
  505,
  404,
  359,
  310,
  337,
  360,
  342,
  406,
  396,
  420,
  472,
  548,
  559,
  463,
  407,
  362,
  405,
  417,
  391,
  419,
  461,
  472,
  535,
  622,
  606,
  508,
  461,
  390,

In [54]:
convert_to_arrow("scripts/passengers.arrow", [df['#Passengers'].values.tolist()])

In [17]:
import os

print(os.getcwd())
# check if the file exists
os.path.isfile("./scripts/training/configs/chronos-t5-tiny.yaml")

/mnt/DATA/Storage/Files/1st Year/Project/chronos-forecasting


True

In [8]:
import subprocess

# Define the command as a list of arguments
command = [
    # "CUDA_VISIBLE_DEVICES=0",
    "python", "scripts/training/train.py",
    "--config", "./scripts/training/configs/chronos-t5-tiny.yaml",
    "--model-id", "amazon/chronos-t5-small",
    "--no-random-init",
    "--max-steps", "1000",
    "--learning-rate", "0.001"
]


# Run the command
result = subprocess.run(command, capture_output=True, text=True)

# Print the output and errors, if any
print("Output:", result.stdout)
print("Error:", result.stderr)

Output: {'loss': 0.508, 'grad_norm': 0.8436999917030334, 'learning_rate': 0.0005, 'epoch': 0.5}
{'loss': 0.4184, 'grad_norm': 0.49798548221588135, 'learning_rate': 0.0, 'epoch': 1.0}
{'train_runtime': 16988.7998, 'train_samples_per_second': 1.884, 'train_steps_per_second': 0.059, 'train_loss': 0.46323553466796874, 'epoch': 1.0}

Error: 2024-08-27 16:24:56,660 - /mnt/DATA/Storage/Files/1st Year/Project/chronos-forecasting/scripts/training/train.py - INFO - TF32 format is only available on devices with compute capability >= 8. Setting tf32 to False.
2024-08-27 16:24:56,660 - /mnt/DATA/Storage/Files/1st Year/Project/chronos-forecasting/scripts/training/train.py - INFO - Using SEED: 4276699687
2024-08-27 16:24:56,665 - /mnt/DATA/Storage/Files/1st Year/Project/chronos-forecasting/scripts/training/train.py - INFO - Logging dir: output/run-6
2024-08-27 16:24:56,665 - /mnt/DATA/Storage/Files/1st Year/Project/chronos-forecasting/scripts/training/train.py - INFO - Loading and filtering 1 dataset